In [1]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from email import message_from_string

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

In [6]:
def get_message_body(msg_payload):
    """Get the body of the message"""
    if "parts" in msg_payload:
        for part in msg_payload["parts"]:
            if part["mimeType"] == "text/plain":
                return base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
            elif part["mimeType"] == "text/html":
                return base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
    else:
        return base64.urlsafe_b64decode(msg_payload["body"]["data"]).decode("utf-8")


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels and messages.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("creds.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        results = service.users().labels().list(userId="me").execute()
        labels = results.get("labels", [])
        label_dict = {label["id"]: label["name"] for label in labels}
        start_date_str = "2024/08/01"
        end_date_str = "2024/08/02"
        time_str = "12:00:00"

        start_datetime_str = f"{start_date_str}"
        end_datetime_str = f"{end_date_str}"

        start_datetime_obj = datetime.strptime(start_datetime_str, "%Y/%m/%d")
        end_datetime_obj = datetime.strptime(end_datetime_str, "%Y/%m/%d")

        start_timestamp = int(start_datetime_obj.timestamp())
        end_timestamp = int(end_datetime_obj.timestamp())

        # Create the query string for filtering messages by date and time
        query = f"after:{start_timestamp} before:{end_timestamp}"

        # Get the list of messages with the specified query
        messages_result = (
            service.users().messages().list(userId="me", q=query).execute()
        )
        messages = messages_result.get("messages", [])

        if not messages:
            print("No messages found.")
            return

        print("\nMessages:")
        for msg in messages:
            msg_id = msg["id"]
            msg_content = (
                service.users().messages().get(userId="me", id=msg_id).execute()
            )
            msg_snippet = msg_content.get("snippet", "No snippet available")
            msg_payload = msg_content.get("payload", {})
            headers = msg_payload.get("headers", [])
            print(headers)
            thread_id = msg_content.get("threadId", "No thread ID")
            subject = "No Subject"
            from_address = "No From address"
            to_address = "No To address"
            internal_date = int(msg_content.get("internalDate", 0)) / 1000
            date = datetime.fromtimestamp(internal_date).strftime("%Y-%m-%d %H:%M:%S")
            label_ids = msg_content.get("labelIds", [])
            labels = [
                label_dict.get(label_id, "Unknown label") for label_id in label_ids
            ]
            for header in headers:
                if header["name"] == "Subject":
                    subject = header["value"]
                elif header["name"] == "From":
                    from_address = header["value"]
                elif header["name"] == "To":
                    to_address = header["value"]

            print(f"Message ID: {msg_id}")
            print(f"Thread ID: {thread_id}")
            print(f"Date: {date}")
            print(f"Labels: {labels}")
            print(f"Subject: {subject}")
            print(f"From: {from_address}")
            print(f"To: {to_address}")
            print(f"Snippet: {msg_snippet}")
            print("Body:")
            print(get_message_body(msg_payload))
            print("=" * 50)

    except HttpError as error:
        # Handle errors from gmail API.
        print(f"An error occurred: {error}")

In [7]:
main()


Messages:
[{'name': 'Delivered-To', 'value': 'subhadeepdoublecap@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:7412:7914:b0:11b:e036:5cfd with SMTP id fa20csp1382929rdb;        Thu, 1 Aug 2024 11:19:22 -0700 (PDT)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IGwx430dYy8ZgmWax0eLdAm+CVLa44FW39J0bucthbVOm8OenRRd/omAvYFtfARPDH6VnMw'}, {'name': 'X-Received', 'value': 'by 2002:a17:90a:8d18:b0:2c9:6278:27c9 with SMTP id 98e67ed59e1d1-2cff95405a9mr1274989a91.38.1722536362537;        Thu, 01 Aug 2024 11:19:22 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1722536362; cv=none;        d=google.com; s=arc-20160816;        b=gWjxl4F2lbCc+5ZCqVtSnbhx1UV0WdE6biUvxDuRop3dGqJUSOgPr8A7SqLew+2HTs         LMPzGCNq12GMdwNUU7krQaV6tJsEYoT+rfbHiJFyF1ROIbVaE4lFJIuelNfX7vRjOpcW         F39lkg8wVfCw+MhX+cmIl+17OLKqKJU2o4zKtM+V3zy4tRHbu+nTZs2Y/5NkyVOLyjJg         s52gFM9qasX8xackPq9wxI7CGEmaZheksUyoe/0jK5VNb33HhrWUib1xgDjYiNeaZeCF         0pcSCGFmFV+HVc6oi2zOcc/GFU26hMb1meO1

In [2]:
import yaml

with open("src/config/config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from src.database.supabase_client import SupabaseClient

client = SupabaseClient()

In [5]:
client.create_table("email", config=config)

In [4]:
from src.core.create_sql_query import create_table_query

create_table_query("email", config["database"]["email"]["columns"])

'CREATE TABLE IF NOT EXISTS public.email (_id varchar primary key,msg_id varchar ,thread_id varchar ,timestamp varchar ,labels varchar ,subject varchar ,from_add varchar ,to_add varchar ,snippet varchar ,file_id varchar );'

In [3]:
config

{'database': {'email': {'columns': [{'col_name': '_id',
     'type': 'varchar',
     'role': 'primary key'},
    {'col_name': 'msg_id', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'thread_id', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'timestamp', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'labels', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'subject', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'from_add', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'to_add', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'snippet', 'type': 'varchar', 'role': 'None'},
    {'col_name': 'file_id', 'type': 'varchar', 'role': 'None'}]},
  'file': 'None',
  'user': 'None'}}